# Scraper til Ekstra-bladet

Scraper til politik-sektionen af eb: https://ekstrabladet.dk/nyheder/politik/

Nedenstående kode blev udarbejdet i forbindelse med CALDISS workshop 5. februar 2021.

## "Design" af scraperen: Hvad skal scrapes?

- [x] Overskrifter
- [x] Links til artikler
- [x] Kildekode til artikel
- [x] Publikationsdato (og tid) for artiklen
- [x] Eksterne links i artiklen
- [x] Antal kommentarer til artiklen

### Andre interessante datapunkter

- Indsamlingstidspunkt (dato og tid)

### Parametre

- Undgå mest læste sektion i højre side
- Undgå generelle nyheder i toppen af side

### "Nice to have"

- Nøgleordssøgning
- Redigeringstidspunkter

## Pakker brugt i scraperen

Der bruges følgende pakker:

**Web scraping**
- `requests`: Tilgå hjemmesider (https://2.python-requests.org/en/master/)
- `BeautifulSoup`: Behandling af HTML-kode (https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- `urljoin` fra `urllib.parse`: Til sammensætning af URL's

**Databehandling**
- `pandas`: Konvertering til tabeldata (https://pandas.pydata.org/)
- `numpy`: Talbehandling og missing
- `re`: Brug af regular expression til at søge og erstatte tekstmønstre

**Tid**
- `datetime` fra `datetime`: Arbejde med datoer
- `time`: Forsinkelser i script

In [21]:
import requests
from bs4 import BeautifulSoup as bs # Funktionen importeres med forkortelsen bs
from urllib.parse import urljoin

import numpy as np
import pandas as pd
import re

from datetime import datetime
import time

## Indlæsning af hjemmeside

Vi starter med indlæsning af hjemmesiden: 'https://ekstrabladet.dk/nyheder/politik/'

Hjemmesiden tilgås ved at sende en GET request (`requests.get()`). Typen af objekt, der returneres, er et `response` objekt, som vi her kalder `response`.

Et `response` objekt indeholder sidens kildekode i attribut `.content`. Dette lagres som separat objekt (`eb_html`).

Til sidst konverteres HTML-koden til et `soup` objekt (`ebsoup`) med funktionen `bs` (`BeautifulSoup`-funktionen importeret med forkortelse). Dette gør HTML naviger- og søgbar. Argumentet `"html.parser"` fortæller, at koden, som skal behandles, er HTML.

In [2]:
response = requests.get('https://ekstrabladet.dk/nyheder/politik/')

eb_html = response.content

ebsoup = bs(eb_html, "html.parser")

Man kan evt. printe HTML-strukturen med nedenstående kommando (dog et voldsomt print).

Funktionen er her "kommenteret ud" med brug af `#`. `#` fortæller Python, at koden er kommentar, som skal ignoreres.

In [3]:
# print(ebsoup.prettify())

## Scrape af links og overskrifter

Scraping af links og overskrifter opnås ved at finde et tag eller en sti af tags, der unikt identificerer de dele af siden, som indeholder links og overskrifter.

Ved at bruge "inspector tool" i en browsers udviklerværktøjer (tilgås typisk med `F12` eller `CTRL + SHIFT + i`), kan vi identificere specifikke HTML tags på siden.

Gennem brug af "inspector tool" erfares, at både links og overskrifter findes under "a" tags ("a" tags bruges altid til links). Derudover erfares, at overskrifterne befinder sig i et "div" tag med klassen "flex flex-wrap--wrap flex-justify--between".

Ved først at selektere den specifikke sektion sikrer vi, at vi ikke henter andre links og overskrifter end dem, som er relevante.

Vi bruger metoden `.find()` til at isolere det specifikke tag. Det, som returneres, er et nyt `soup` objekt (`section_soup`), som vi også kan søge i.

In [4]:
section_soup = ebsoup.find('div', class_ = 'flex flex-wrap--wrap flex-justify--between') # Sektion uden mest læste og top

Når sektionen er isoleret, kan vi søge i dette `soup` objekt efter links og overskrifter ("a" tags)

In [5]:
link_soups = section_soup.find_all('a') # Links

"a" tags indeholder både overskriften og link til artiklen.

Overskriften befinder sig i et "h2" tag. For at trække overskriften ud, er vi derfor nødt til at isolere "h2"-tagget i hver "a" tag (`headline_soups`). Derefter hentes selve teksten for hvert tag (`headlines`).

Et "for loop" bruges her til at gå igennem hvert "a" tag (`link_soups`) og trække "h2"-taggenen ud. Derefter bruges et "for loop" igen til at tage teksten ud af hvert "h2" tag.

In [9]:
# Headlines - omstændig

headline_soups = []

for link_soup in link_soups:
    headline_soups.append(link_soup.find('h2'))

headlines = []

for headline_soup in headline_soups:
    headlines.append(headline_soup.get_text(strip = True))
    
headlines

['Erhvervsminister Kollerup er tilbage fra sygeorlov',
 'Coronahelte får skattesmæk: Nu mødes de om løsning',
 "USA's tidligere udenrigsminister George Shultz er død",
 'Nye Borgerlige-profil anklaget for hykleri',
 'Engell: Bramsens fjender lugter blod',
 'Marzouk fastholder anklager mod Khader',
 'Fortidens spøgelser hjemsøger Clinton-familien',
 'Franciska Rosenkilde er Alternativets nye leder',
 'Anmeldt for overgreb: Politiet stopper efterforskning af Khader',
 'Usmagelig madmanøvre fra Mette Makrel',
 'Databrud: Styrelse lækker 150.000 cpr-numre',
 'Trump presset ud af fagforening',
 'USA vil fjerne Yemens Houthi-bevægelse fra terrorliste igen',
 'EL vil bruge milliard på trivsel: Flere lærere og lejrskoler',
 '1000 deltager i største protest mod militærkup i Myanmar',
 'Mette F. frygter vaccine-konflikt',
 'Ekspert om aggressiv retorik mellem USA og Rusland: En overreaktion',
 'Elever skal til færre eksaminer i skoler og gymnasier',
 'Ekstra Bladet går mod-live på Mette F.',
 'N

Til links gøres igen brug af et "for loop". Links ligger under attribut `href` i "a" tagget. Attributter i tags kan tilgås med `[]`.

In [10]:
links = []

for link_soup in link_soups:
    links.append(link_soup['href'])
    
links

['/nyheder/politik/erhvervsminister-kollerup-er-tilbage-fra-sygeorlov/8465480',
 '/nyheder/politik/danskpolitik/coronahelte-faar-skattesmaek-nu-moedes-de-om-loesning/8464345',
 '/nyheder/politik/usas-tidligere-udenrigsminister-george-shultz-er-doed/8465016',
 '/nyheder/politik/danskpolitik/nye-borgerlige-profil-anklaget-for-hykleri/8464639',
 '/nyheder/politik/engell-bramsens-fjender-lugter-blod/8464625',
 '/nyheder/politik/danskpolitik/marzouk-fastholder-anklager-mod-khader/8464850',
 '/nyheder/politik/fortidens-spoegelser-hjemsoeger-clinton-familien/8444065',
 '/nyheder/politik/danskpolitik/franciska-rosenkilde-er-alternativets-nye-leder/8464660',
 '/nyheder/politik/danskpolitik/anmeldt-for-overgreb-politiet-stopper-efterforskning-af-khader/8464779',
 '/nyheder/politik/danskpolitik/usmagelig-madmanoevre-fra-mette-makrel/8464307',
 '/nyheder/politik/danskpolitik/databrud-styrelse-laekker-150.000-cpr-numre/8464199',
 '/nyheder/politik/trump-presset-ud-af-fagforening/8463057',
 '/nyhede

Ovenstående koder kan kondenseres på flere måder.

For det første tillader for loops, at der er flere argumenter. Vi kan derfor køre et for loop, hvor vi både tilføjer til links-listen og headlines-listen.

Derudover understøtter Python at man kæder kommandoer sammen. Når man bruger metoder (`.find()` og `.get_text()` er fx metoder), tjekker Python om metoden er kompatibel med typen af objekt, som det bruges på. Derudover returnerer en metode typisk en eller anden form for objekt. Man kan derfor anvende en metode direkte i direkte forlængelse af en anden metode, da Python ved, at den anden metode så skal anvendes på outputtet af den første:

In [11]:
# Links og headlines - kondenseret

links = []
headlines = []

for link_soup in link_soups:
    links.append(link_soup['href'])
    headlines.append(link_soup.find('h2').get_text(strip = True))
    
print(links)
print(headlines)

['/nyheder/politik/erhvervsminister-kollerup-er-tilbage-fra-sygeorlov/8465480', '/nyheder/politik/danskpolitik/coronahelte-faar-skattesmaek-nu-moedes-de-om-loesning/8464345', '/nyheder/politik/usas-tidligere-udenrigsminister-george-shultz-er-doed/8465016', '/nyheder/politik/danskpolitik/nye-borgerlige-profil-anklaget-for-hykleri/8464639', '/nyheder/politik/engell-bramsens-fjender-lugter-blod/8464625', '/nyheder/politik/danskpolitik/marzouk-fastholder-anklager-mod-khader/8464850', '/nyheder/politik/fortidens-spoegelser-hjemsoeger-clinton-familien/8444065', '/nyheder/politik/danskpolitik/franciska-rosenkilde-er-alternativets-nye-leder/8464660', '/nyheder/politik/danskpolitik/anmeldt-for-overgreb-politiet-stopper-efterforskning-af-khader/8464779', '/nyheder/politik/danskpolitik/usmagelig-madmanoevre-fra-mette-makrel/8464307', '/nyheder/politik/danskpolitik/databrud-styrelse-laekker-150.000-cpr-numre/8464199', '/nyheder/politik/trump-presset-ud-af-fagforening/8463057', '/nyheder/politik/us

Med funktionen `len()` kan vi tjekke, om vi har hentet det samme antal links og headlines.

In [13]:
print(len(links),
      len(headlines),
     sep = "\n")

20
20


## Artikel-scrape

Scraperen skal kunne hente følgende indhold for en artikel:

- Kildekode
- Dato
- Antal kommentarer
- Eksterne links
- Forfatter

Vi starter først med at finde de relevante tags for en artikel. Derefter dannes en funktion ud fra dette, som anvendes på alle artikler. Baseret på de fejl, der opstår, når funktionen kører på alle artikler, tilrettes funktionen.

### Kildekode

Vi starter med at lave et soup objekt for en af artiklerne. Da soup objektet netop indeholder kildekoden (HTML), har vi dermed styr på første del, som scrape af artiklen skal indeholde:

In [14]:
article_url = "https://ekstrabladet.dk/nyheder/politik/danskpolitik/skal-grilles-om-skandale-interview/8463225"

In [15]:
response = requests.get(article_url)

html = response.content

artsoup = bs(html, "html.parser")

### Dato

Der er to måder at indhente dato og tidspunkt. Dato og tidspunkt indgår både i selve hjemmeside indholdet i toppen af artiklen ("span" tag med klassen "article-timestamp--top"):

In [16]:
# date - solution 1
artdate = artsoup.find('span', class_ = "article-timestamp--top").get_text(strip = True)
print(artdate)

Fredag  d. 5. feb. 2021 - kl. 10:59


Derudover findes dato og tidspunkt også som en del af metadata i HTML'en (conten-attribut i tag "meta" med property-attribute "og:article:published_time"):

In [18]:
# date - solution 2
artdate2 = artsoup.find('meta', attrs = {'property': 'og:article:published_time'})['content']
print(artdate2)

2021-02-05T09:59:33Z


Der virker til at være en uoverenstemmelse mellem de to tidspunkter. Derfor lagres begge, så man senere kan undersøge, hvilken der giver bedst mening at bruge.

### Antal kommentarer

Antal kommentarerer findes under "span" tag med id "fnTalkCommentText".

In [24]:
# n comments
ncomments = artsoup.find('span', id = 'fnTalkCommentText').get_text()
print(ncomments)

313 kommentarer


Da vi blot er interesseret i antal kommentarer, kan vi konverterer det til et heltal (integer).

Dette gøres ved først at fjerne teksten " kommentarer" og konverterer objektet til et heltal.

Teksten fjernes gennem brug af regular expression (`re`). Regular expressions tillader, at man søger efter mønstre i tekst, som man derefter kan erstatte. I regular expression betyder "\s" fx "whitespace" (fx et mellemrum), "." betyder "enhver karakter" og "*" betyder "gentag 0 eller flere gange". Et regular expression `"\s.*"` matcher derfor tekststykker bestående af et mellemrum efterfulgt af 0 eller flere karakterer.

Tekststykker der matcher et regular expression kan erstattes med `re.sub()`. I funktionen skrives først mønstret, som skal erstattes (`'\s.*'`, dernæst det, som det skal erstattes med (`''` for ingenting) og til sidst teksten, hvor der skal erstattes (`ncomments`):

Tallet, som står tilbage i teksten, konverteres til heltal med `int()`.

In [23]:
ncomments_int = int(re.sub(r'\s.*', '', ncomments))
print(ncomments_int)

313


### Eksterne links

Links er som bekendt altid lagret i "a" tags". For at finde links til andre sider i artikelteksten, skal vi derfor have indskrænket til selve artikelteksten, før der søges efter links.

Selve artikelteksten findes i et "div" tag med klassen "article-bodytext".

In [26]:
# external links

body_soup = artsoup.find('div', class_ = 'article-bodytext')

extlink_soups = body_soup.find_all('a')

extlinks = []
for extlink_soup in extlink_soups:
    extlinks.append(extlink_soup['href'])

print(extlinks)

['https://olfi.dk/2021/01/27/saadan-koerte-rutineret-journalists-interview-med-forsvarsministeren-helt-af-sporet/', 'https://nyheder.tv2.dk/politik/2021-02-04-forsvarsministeren-beskylder-journalist-for-at-opdigte-skandaleinterview']


Ovenstående kode kan forkortes gennem brug af såkaldt "list comprehension" (https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions).

"List comprehension" gør, at man kombinere et for loop og en liste sådan, at man én gang laver en liste, som består af outputs fra et for loop:

In [28]:
# external links - short version
extlinks = [extlink_soup['href'] for extlink_soup in artsoup.find('div', class_ = 'article-bodytext').find_all('a')]

print(extlinks)

['https://olfi.dk/2021/01/27/saadan-koerte-rutineret-journalists-interview-med-forsvarsministeren-helt-af-sporet/', 'https://nyheder.tv2.dk/politik/2021-02-04-forsvarsministeren-beskylder-journalist-for-at-opdigte-skandaleinterview']


### Forfatter

Forfatteren findes umiddelbart i et "span" tag med itemprop-attribut med væriden "author":

In [29]:
# author
author = artsoup.find('span', attrs = {'itemprop': 'author'}).get_text(strip = True)
print(author)

James Kristoffer Miles


## Artikelscraper som funktion

Nu vi ved, hvordan de forskellige dele hentes, kan vi konverterer dette til en funktion.

Vi skriver funktionen sådan, at den tager et link som input/argument (link til artikel). Funktionen tilgår derefter artiklen og henter de relevante informationerne.

Informationerne returneres som en såkaldt "dictionary" (`article_dict`). En dictionary består af nøgle-værdi par, hvor hver værdi tildeles en vis nøgle. Nøgler i dictionaries kan anvendes lig variable i et datasæt.

**Fejlhåndtering**

Koden skrevet indtil videre antager, at informationerne altid er der (fx forfatter, antal kommentarer, eksterne links). Det er dog meget sandsynligt, at nogen artikler ikke indeholder alle disse informationer. Funktionen skal derfor kunne tage højde for dette.

I funktionen tages højde for dette gennem brug brug af simpel fejlhåndtering med brug af `try` og `except`. `try-except` tillader, at man tager højde for visse fejl i funktionen, sådan at funktionen ikke stopper, hvis den rammer den fejl.

Man skriver først en `try`-blok. I den specificerer man den kode, som Python skal forsøge at køre. Man skriver derefter en `except`-blok. `except`-linjen skal gerne indeholde den fejltype, som man forventer at `try`-blokken vil støde på (i dette tilfælde en `AttributeError`). Koden i `except`-blokken køres så i det tilfælde, at den specificerede fejltype sker, når `try`-blokken køres.

Sagt kort: Forsøg at kør `try`-koden. Hvis der opstår en `AttributeError`, kør da `except`-koden.

Koden kunne sagtens forbedres til at tage højde for flere ting, men her nøjes vi med at forsøge en ting og ellers give en tom værdi, hvis ikke det lykkes. 

**Obs: Tilgå URL**

Der kan opstå mange fejl, når man tilgår links/URL's i Python. Fx kan der nemt opstå en mindre forbindelsesfejl som gør, at man ikke får forbindelse i lige det øjeblik, som man kører koden. 

Derfor vil man typisk skrive funktioner, som skal få adgang til links, sådan, at de fx forsøger igen eller forsøger noget andet, hvis de støder på nogen bestemte fejltyper.

Vær opmærksom på , at vi i denne funktion *ikke* har taget højde for dette!

In [30]:
def eb_article_scrape(url):
    article_dict = {}
    
    response = requests.get(url)
    html = response.content
    artsoup = bs(html, "html.parser")
    
    artdate = artsoup.find('span', class_ = "article-timestamp--top").get_text(strip = True)
    artdate2 = artsoup.find('meta', attrs = {'property': 'og:article:published_time'})['content']
    
    try:
        ncomments = artsoup.find('span', id = 'fnTalkCommentText').get_text()
        ncomments_int = int(re.sub(r'\s.*', '', ncomments))
    except AttributeError: 
        ncomments_int = np.nan
    
    try:
        extlinks = [extlink_soup['href'] for extlink_soup in artsoup.find('div', class_ = 'article-bodytext').find_all('a')]
    except AttributeError:
        extlinks = ""
    
    try:
        author = artsoup.find('span', attrs = {'itemprop': 'author'}).get_text(strip = True)
    except AttributeError:
        author = ""
    
    article_dict['source_code'] = str(artsoup)
    article_dict['artdate_printed'] = artdate
    article_dict['artdate_meta'] = artdate2
    article_dict['ncomments'] = ncomments_int
    article_dict['extlinks'] = extlinks
    article_dict['author'] = author
    article_dict['article_url'] = url
    
    return(article_dict)

Når man laver en funktion med `def` defineres funktionen blot. Den er derfor ikke kørt, men er oprettet, så vi kan anvende den.

I nedenstående bruges funktionen på samme artikel som før. Output (dictionary med informationerne) lagres i objektet `article`.

In [31]:
article = eb_article_scrape(article_url)

Da objektet indeholder kildekoden, vil det fylde en del at printe hele dicionary. Vi kan dog tjekke, hvilke "nøgler" dicionary indeholder med `.keys()`:

In [33]:
article.keys()

dict_keys(['source_code', 'artdate_printed', 'artdate_meta', 'ncomments', 'extlinks', 'author', 'article_url'])

Vi kan tilgå en af værdierne ved at specificere nøglen i kantede parenteser (`[]`):

In [34]:
article['ncomments']

313

## Anvendelse af funktionen på flere links

Når funktionen nu er dannet, kan denne bruges på de indsamlede links.

Da de fleste links er hentet som relative links (dvs. links, som tager udgangspunkt i hoveddomænet/hovedsiden), så er vi nødt til at tilføje hoveddomænet hovedsiden til hvert enkelt link.

Dette gøres med et for loop. Samtidig bruges funktionen `urljoin()` fra `urllin.parse`, da denne tager højde for, at hoveddomæne/hovedside ikke er en del af linket i forvejen:

In [35]:
new_links = [urljoin("https://ekstrabladet.dk", link) for link in links]
new_links

['https://ekstrabladet.dk/nyheder/politik/erhvervsminister-kollerup-er-tilbage-fra-sygeorlov/8465480',
 'https://ekstrabladet.dk/nyheder/politik/danskpolitik/coronahelte-faar-skattesmaek-nu-moedes-de-om-loesning/8464345',
 'https://ekstrabladet.dk/nyheder/politik/usas-tidligere-udenrigsminister-george-shultz-er-doed/8465016',
 'https://ekstrabladet.dk/nyheder/politik/danskpolitik/nye-borgerlige-profil-anklaget-for-hykleri/8464639',
 'https://ekstrabladet.dk/nyheder/politik/engell-bramsens-fjender-lugter-blod/8464625',
 'https://ekstrabladet.dk/nyheder/politik/danskpolitik/marzouk-fastholder-anklager-mod-khader/8464850',
 'https://ekstrabladet.dk/nyheder/politik/fortidens-spoegelser-hjemsoeger-clinton-familien/8444065',
 'https://ekstrabladet.dk/nyheder/politik/danskpolitik/franciska-rosenkilde-er-alternativets-nye-leder/8464660',
 'https://ekstrabladet.dk/nyheder/politik/danskpolitik/anmeldt-for-overgreb-politiet-stopper-efterforskning-af-khader/8464779',
 'https://ekstrabladet.dk/nyhe

### Scrape af alle artikler

Vi har nu både listen af links og vores scrape-funktion, og vi er nu klar til at scrape alle artiklerne.

De enkelte links scrapes med brug af for loop. Artiklerne lagres som en liste (`articles`). Den færdige datastruktur er altså en liste bestående af dictionaries. Hver dictionary indeholder de forskellige informationer fra artiklen.

***OBS:*** Med brug af for loops og scrape skal man være varsom. Hvis ikke man indbygger forsinkelser eller andet, så sender man rigtig mange henvendelser til en hjemmeside på næsten ingen tid - det kan dem, som ejer hjemmesiden ikke lide!

Nedenståedne kode har indbyggede forsinkelser med `time.sleep()`. Denne funktion sætter koden på pause i det antal sekunder, som man sætter ind. I dette tilfælde venter Python altså 2 sekunder mellem hvert scrape.

(jeg har desuden sat et mellemrum i `for`-linjen, så man ikke bare kører denne linje uden at læse efter først).

In [37]:
import time

articles = []

for c, link in enumerate(new_links, start = 1):
    articles.append(eb_article_scrape(link))
    
    print("{:.2f}%".format(100.0 * c/len(new_links)), end = '\r')
    
    time.sleep(2) #VIGTIGT! Denne kommando gør, at der ventes to sekunder mellem hvert scrape

### Tilføj overskrift og link til data

Funktionen tilføjer lige nu ikke overskrift og link. Man kunne skrive funktionen om, så den gjorde dette.

I dette tilfælde tilføjer vi overskrift og link til listen `articles` ved at gå igennem hver dictionary i listen med et for loop, og tilføje den tilhørende overskrift og link. Dette gøres ved at loope igennem indeks i både artikel-listen (`articles`), overskrift-listen (`headlines`) og link-listen (`new_links`).

Da de har samme rækkefølge, kan man loope igennem dem på denne måde (første artikel i artikellisten (`articles[0]`) hører til første headline og link (hhv. `headlines[0]` og `new_links[0]`) osv.).

In [99]:
for i in range(0, len(articles)-1):
    articles[i]['headline'] = headlines[i]
    articles[i]['url'] = new_links[i]

### Konvertering til tabel (data frame)

Dataen, som er indsamlet, kan konverteres til en tabelstruktur. Dette gøres gennem pakken `pandas`, hvor man kan arbejde med datastrukturen "data frame".

Funktionen `pd.DataFrame.from_records()` danner en data frame fra en datastruktur som den, vi har indsamlet artiklerne som (en liste af dictionaries).

In [38]:
import pandas as pd

data = pd.DataFrame.from_records(articles)

data.head()

,source_code,artdate_printed,artdate_meta,ncomments,extlinks,author,article_url
0,"<!DOCTYPE html>\n\n<html lang=""da"">\n<head>\n<...",Mandag d. 8. feb. 2021 - kl. 08:11,2021-02-08T07:11:29Z,139.0,[],,https://ekstrabladet.dk/nyheder/politik/erhver...
1,"<!DOCTYPE html>\n\n<html lang=""da"">\n<head>\n<...",Mandag d. 8. feb. 2021 - kl. 06:55,2021-02-08T05:55:58Z,145.0,[https://ekstrabladet.dk/nyheder/politik/dansk...,Kristian B. Larsen,https://ekstrabladet.dk/nyheder/politik/danskp...
2,"<!DOCTYPE html>\n\n<html lang=""da"">\n<head>\n<...",Søndag d. 7. feb. 2021 - kl. 19:18,2021-02-07T18:18:33Z,16.0,[],,https://ekstrabladet.dk/nyheder/politik/usas-t...
3,"<!DOCTYPE html>\n\n<html lang=""da"">\n<head>\n<...",Søndag d. 7. feb. 2021 - kl. 19:12,2021-02-07T18:12:31Z,1463.0,[https://ekstrabladet.dk/nyheder/politik/afsae...,Emmely Smith,https://ekstrabladet.dk/nyheder/politik/danskp...
4,"<!DOCTYPE html>\n\n<html lang=""da"">\n<head>\n<...",Søndag d. 7. feb. 2021 - kl. 18:26,2021-02-07T17:26:19Z,NaN,[],Hans Engell,https://ekstrabladet.dk/nyheder/politik/engell...


## Sæt det hele sammen

Nedenstående kondenserer koden til det mest essentielle.

Som det forhåbentlig kan ses, er der i sidste ende ikke tale om forfærdelig meget kode, for at være godt på vej til en scraper:

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import numpy as np
import time
from urllib.parse import urljoin
import pandas as pd

In [ ]:
response = requests.get('https://ekstrabladet.dk/nyheder/politik/')

eb_html = response.content

ebsoup = bs(eb_html, "html.parser")

In [ ]:
section_soup = ebsoup.find('div', class_ = 'flex flex-wrap--wrap flex-justify--between') # Sektion uden mest læste og top
link_soups = section_soup.find_all('a') # Links

In [ ]:
# Links og headlines - kondenseret

links = []
headlines = []

for link_soup in link_soups:
    links.append(link_soup['href'])
    headlines.append(link_soup.find('h2').get_text(strip = True))

In [ ]:
def eb_article_scrape(url):
    article_dict = {}
    
    response = requests.get(url)
    html = response.content
    artsoup = bs(html, "html.parser")
    
    artdate = artsoup.find('span', class_ = "article-timestamp--top").get_text(strip = True)
    artdate2 = artsoup.find('meta', attrs = {'property': 'og:article:published_time'})['content']
    
    try:
        ncomments = artsoup.find('span', id = 'fnTalkCommentText').get_text()
        ncomments_int = int(re.sub(r'\s.*', '', ncomments))
    except AttributeError: 
        ncomments_int = np.nan
    
    try:
        extlinks = [extlink_soup['href'] for extlink_soup in artsoup.find('div', class_ = 'article-bodytext').find_all('a')]
    except AttributeError:
        extlinks = ""
    
    try:
        author = artsoup.find('span', attrs = {'itemprop': 'author'}).get_text(strip = True)
    except AttributeError:
        author = ""
    
    article_dict['source_code'] = str(artsoup)
    article_dict['artdate_printed'] = artdate
    article_dict['artdate_meta'] = artdate2
    article_dict['ncomments'] = ncomments_int
    article_dict['extlinks'] = extlinks
    article_dict['author'] = author
    article_dict['article_url'] = url
    
    return(article_dict)

In [ ]:
new_links = [urljoin("https://ekstrabladet.dk", link) for link in links]

In [ ]:
articles = []

f or c, link in enumerate(new_links, start = 1):
    articles.append(eb_article_scrape(link))
    
    print("{:.2f}%".format(100.0 * c/len(new_links)), end = '\r')
    
    time.sleep(2)

In [ ]:
data = pd.DataFrame.from_records(articles)